In [2]:
# !pip install --upgrade pip
# !pip install gensim
# !pip install nltk
# !pip install tokenizers
# !pip install sentencepiece
# !pip install python-bidi
# !pip install arabic-reshaper
# # !pip install PyArabic
# !pip install spacy

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms import ToTensor
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import pandas as pd
# from pyarabic.araby import tokenize, strip_tashkeel
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np
from utilities import *
import textProcessing as tp
import nltk
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec
import spacy
# nltk.download('punkt')
# train_text = load_text("dataset/train.txt")
# tp.preprocessing_text(train_text,"train_preprocessed.txt")


DIACRITIC2INDEX:  dict_items([('ً', 0), ('ٌ', 1), ('ٍ', 2), ('َ', 3), ('ُ', 4), ('ِ', 5), ('ّ', 6), ('ًّ', 7), ('ٌّ', 8), ('ٍّ', 9), ('َّ', 10), ('ُّ', 11), ('ِّ', 12), ('ْ', 13), ('', 14), ('0', 15)])
{'ً': 0, 'ٌ': 1, 'ٍ': 2, 'َ': 3, 'ُ': 4, 'ِ': 5, 'ّ': 6, 'ًّ': 7, 'ٌّ': 8, 'ٍّ': 9, 'َّ': 10, 'ُّ': 11, 'ِّ': 12, 'ْ': 13, '': 14, '0': 15}
16
{'ء': 0, 'آ': 1, 'أ': 2, 'ؤ': 3, 'إ': 4, 'ئ': 5, 'ا': 6, 'ب': 7, 'ة': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ك': 28, 'ل': 29, 'م': 30, 'ن': 31, 'ه': 32, 'و': 33, 'ى': 34, 'ي': 35, ' ': 36, '0': 37, 's': 38}
39


In [4]:
print(torch.__version__)

2.1.2+cu118


In [5]:
# Download via terminal commands
# !wget "https://archive.org/download/aravec2.0/tweet_cbow_300.zip"
# !unzip "tweet_cbow_300.zip"

In [6]:
# # # make a directory called "spacyModel"
# # %mkdir spacyModel
# # model = gensim.models.Word2Vec.load("tweets_cbow_300")
# # # export the word2vec fomart to the directory
# # model.wv.save_word2vec_format("./spacyModel/aravec.txt")
# # # using `gzip` to compress the .txt file
# # !gzip ./spacyModel/aravec.txt
# # !python -m spacy init vectors ar ./spacyModel/aravec.txt.gz spacy.aravec.model
nlp = spacy.load("./spacy.aravec.model/")

In [7]:
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec
from torch import mode

class AraVecEmbbedding():
    def __init__(self,model):
        self.model=model

    def map_words_to_vectors(self,sentence):
        return [(token.text,token.vector) for token in self.model(sentence)]


    def AraVec_wordEmbedding(self,sentence):
        return self.map_words_to_vectors(sentence)


In [8]:
from utilities import *
from torch.utils.data import Dataset, DataLoader
import textProcessing as tp
import torch
import numpy as np
from data_preprocessing import DataPreprocessing
import re
class MyDataset(Dataset):
    def __init__(self,model,dataset_path="dataset/train_preprocessed.txt",T=100):
        self.dataPreprocessor = DataPreprocessing()
        self.T = T
        self.data = load_text(dataset_path)
        self.wordEmbedding=AraVecEmbbedding(model)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        sentence = self.data[idx]
        # extract the label
        labels,sentence= self.dataPreprocessor.extract_diacritics_with_previous_letter('s'+sentence)

        # check sentence length
        if len(sentence) > self.T:
            sentence = sentence[:self.T]
            labels = labels[:self.T]
        else:
            for i in range(self.T - len(sentence)):
                sentence += '0'
                labels.append('0')

        assert len(sentence) == self.T
        assert len(labels) == self.T

        # convert the sentence to embedding
        # Word
        arabic_words_with_spaces =  re.findall(r'\s*\b\w+\b\s*|,\s*|\s*\b\w+\b(?!\s*$)',sentence)
        sentence_char = self.dataPreprocessor.convert_sentence_to_vector(sentence)
        sentence_word_embedding =self.wordEmbedding.AraVec_wordEmbedding(sentence)
        list_of_char = []
        k=0
        for i, (word, word_embedding) in enumerate(sentence_word_embedding):
            length_arabic = len(arabic_words_with_spaces[i])
            for j in range(k,k+length_arabic):
                li = sentence_char[j].astype(float)
                concatenated_array = np.concatenate((li, np.array(word_embedding)), axis=0)
                list_of_char.append(concatenated_array)
            k+=length_arabic
        assert len(list_of_char) == self.T
        assert len(labels) == self.T
        # convert the labels to one hot encoding
        labels = self.dataPreprocessor.convert_labels_to_indices(labels)

        # reshape the labels
        labels = labels.reshape(-1,1)

        # convert the sentence and labels to tensors
        sentence = torch.tensor(list_of_char, dtype=torch.float32)
        labels = torch.LongTensor(labels)

        return sentence, labels


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from LSTM import LSTMClassifier
from data_preprocessing import DataPreprocessing

class LSTMTrainer:
    def __init__(self,load=True,epoch = 0,input_size = 39,hidden_size = 128,output_size = 16,batch_size = 512,num_epochs = 20):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.model = LSTMClassifier(self.input_size, self.hidden_size, self.output_size)
        self.current_epoch = 0
        self.current_epoch = epoch
        if load:
            self.load_model(epoch)
        self.model.to(self.device)
        self.dataset = MyDataset(model = nlp,T = 280)
        self.test_dataset = MyDataset(model = nlp,dataset_path="dataset/test_preprocessed.txt",T = 280)
        self.train_dataloader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=False)
        self.test_dataloader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)
        self.criterion = nn.CrossEntropyLoss(ignore_index=15)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.scheduler = StepLR(self.optimizer, step_size=2, gamma=0.5)


    def train(self):
        for epoch in range(0,self.current_epoch):
            self.scheduler.step()
        for epoch in range(self.current_epoch,self.num_epochs):
            self.model.train()
            running_loss = 0.0
            for i, data in enumerate(self.train_dataloader, 0):
                # get the inputs
                inputs, labels = data
                # print("input shape",inputs.size())
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # zero the parameter gradients
                self.optimizer.zero_grad()
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                loss = self.criterion(outputs, labels)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)
                self.optimizer.step()
                # print statistics
                running_loss += loss.item()
                if i % 2 == 0:  # print every 100 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2))
                    running_loss = 0.0
                if i%500==499:      
                # save the model if it has the best training loss till now
                    self.save_model(epoch+1)
            self.scheduler.step()

        print('Finished Training')

    def test(self):
        self.model.eval()
        with torch.no_grad():
            running_loss = 0.0
            for i, data in enumerate(self.test_dataloader, 0):
                # get the inputs
                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                loss = self.criterion(outputs, labels)
                running_loss += loss.item()
            print('Test loss: %.3f' %
                  (running_loss / len(self.test_dataloader)))
    def calcluate_accuracy(self):
        self.model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for i, data in enumerate(self.test_dataloader, 0):
                # get the inputs
                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                _, predicted = torch.max(outputs.data, 1)
                      # cut the padding
                predicted = predicted[labels != 15]
                labels = labels[labels != 15]
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print('Accuracy of the network on the test set: %d %%' % (
                    100 * correct / total))
            print('Accuracy of the network on the test set: %f %%' % (
                    100 * correct / total))
        with torch.no_grad():
            correct = 0
            total = 0
            for i, data in enumerate(self.train_dataloader, 0):
                # get the inputs
                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                # forward + backward + optimize
                outputs = self.model(inputs)
                outputs = outputs.view(-1, outputs.shape[-1])
                labels = labels.view(-1)
                _, predicted = torch.max(outputs.data, 1)
                      # cut the padding
                predicted = predicted[labels != 15]
                labels = labels[labels != 15]
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print('Accuracy of the network on the train set: %d %%' % (
                    100 * correct / total))
            # print floating point accuracy
            print('Accuracy of the network on the train set: %f %%' % (
                    100 * correct / total))
    def save_model(self,epoch):
        torch.save(self.model.state_dict(), "models/lstm_model_word_Embedd"+str(epoch)+".pth")
    def load_model(self,epoch=9):
        self.model.load_state_dict(torch.load("models/lstm_model_"+str(epoch)+".pth"))
        self.model.eval()
        self.current_epoch = epoch



In [23]:
# from LSTM_Training import LSTMTrainer
lstmTrainer = LSTMTrainer(load = False,epoch = 0,input_size = 339,hidden_size = 1024,output_size = 16,batch_size = 16,num_epochs = 5)

In [22]:
# import gc
# import torch
# torch.cuda.empty_cache()
# gc.collect()

0

In [24]:
lstmTrainer.train()

[5,  4235] loss: 0.014
[5,  4237] loss: 0.008
[5,  4239] loss: 0.010
[5,  4241] loss: 0.008
[5,  4243] loss: 0.007
[5,  4245] loss: 0.009
[5,  4247] loss: 0.004
[5,  4249] loss: 0.008
[5,  4251] loss: 0.008
[5,  4253] loss: 0.005
[5,  4255] loss: 0.009
[5,  4257] loss: 0.006
[5,  4259] loss: 0.007
[5,  4261] loss: 0.008
[5,  4263] loss: 0.004
[5,  4265] loss: 0.005
[5,  4267] loss: 0.006
[5,  4269] loss: 0.014
[5,  4271] loss: 0.005
[5,  4273] loss: 0.011
[5,  4275] loss: 0.008
[5,  4277] loss: 0.012
[5,  4279] loss: 0.008
[5,  4281] loss: 0.008
[5,  4283] loss: 0.008
[5,  4285] loss: 0.010
[5,  4287] loss: 0.007
[5,  4289] loss: 0.006
[5,  4291] loss: 0.011
[5,  4293] loss: 0.007
[5,  4295] loss: 0.006
[5,  4297] loss: 0.008
[5,  4299] loss: 0.009
[5,  4301] loss: 0.009
[5,  4303] loss: 0.010
[5,  4305] loss: 0.007
[5,  4307] loss: 0.007
[5,  4309] loss: 0.006
[5,  4311] loss: 0.005
[5,  4313] loss: 0.009
[5,  4315] loss: 0.005
[5,  4317] loss: 0.007
[5,  4319] loss: 0.006
[5,  4321] 

In [25]:
lstmTrainer.test()

Test loss: 0.098


In [26]:
lstmTrainer.calcluate_accuracy()

Accuracy of the network on the test set: 97 %
Accuracy of the network on the test set: 97.723398 %


KeyboardInterrupt: 